## 📦 Setup & Configuration

In [ ]:
# Включаем DEBUG логирование
import os

from app.config import ensure_dotenv
from app.logging_config import configure_logging

os.environ['LOG_LEVEL'] = 'DEBUG'

# Загружаем .env
ensure_dotenv()

# Настраиваем логирование
configure_logging()

print('✅ Logging configured')

✅ Logging configured


In [ ]:
# Проверяем переменные окружения для агентов
env_vars = [
    'GIGACHAT_CREDENTIALS',
    'GIGACHAT_SCOPE',
    'GIGACHAT_MODEL',
    'EMBEDDINGS_MODEL',
    'API_GEO',
    'API_SITE',
    'REGION_ID',
    'LANGSMITH_TRACING',
]

print('🔧 Environment Variables:')
print('=' * 50)
for var in env_vars:
    val = os.getenv(var, 'NOT SET')
    if 'CREDENTIALS' in var or 'API_KEY' in var:
        val = f'{val[:20]}...' if len(val) > 20 else val
    print(f'{var}: {val}')

🔧 Environment Variables:
GIGACHAT_CREDENTIALS: MDE5YWNmM2YtMjFiOC03...
GIGACHAT_SCOPE: GIGACHAT_API_PERS
GIGACHAT_MODEL: GigaChat-2
EMBEDDINGS_MODEL: EmbeddingsGigaR
API_GEO: https://yazzh-geo.gate.petersburg.ru
API_SITE: https://yazzh.gate.petersburg.ru
REGION_ID: 78
LANGSMITH_TRACING: true


---
## 🔤 LLM Configuration

In [ ]:
# Создаём LLM
from app.agent.llm import get_llm

llm = get_llm()
print(f'✅ LLM created: {type(llm).__name__}')
print(f'   Model: {llm.model if hasattr(llm, "model") else "N/A"}')

f:\sber-agent-hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM created: GigaChat
   Model: GigaChat-2-Max


In [ ]:
# Тестовый вызов LLM
from langchain_core.messages import HumanMessage

test_message = HumanMessage(content='Привет! Как дела?')
response = llm.invoke([test_message])

print('🔤 LLM Test:')
print(f'   Input: {test_message.content}')
print(f'   Output: {response.content}')

OAUTH UPDATE TOKEN
Tracing control thread func compress parallel called
Sending multipart request to https://api.smith.langchain.com with context: trace=b402bd4e-eb4c-4728-8ef2-30cf42eb379b,id=b402bd4e-eb4c-4728-8ef2-30cf42eb379b
🔤 LLM Test:
   Input: Привет! Как дела?
   Output: Привет! У меня всё отлично — готов обсудить любые темы, ответить на вопросы или просто пообщаться. А ты как себя чувствуешь сегодня?


---
## 🛡️ Toxicity Filter Testing

In [ ]:
# Загружаем toxicity filter
from app.services.toxicity import ToxicityLevel, get_toxicity_filter

toxicity_filter = get_toxicity_filter()
print('✅ Toxicity filter loaded')

✅ Toxicity filter loaded


In [ ]:
# Sample тексты для тестирования
TOXICITY_SAMPLES = [
    ('Как получить СНИЛС?', False, 'Обычный вопрос'),
    ('Спасибо за помощь!', False, 'Благодарность'),
    ('Блин, ничего не работает', True, 'Лёгкое недовольство'),
    ('Какого хрена?!', True, 'Грубость'),
    ('Это полный бред', True, 'Негатив'),
]

print('🛡️ Toxicity Filter Tests:')
print('=' * 60)

for text, expected_toxic, description in TOXICITY_SAMPLES:
    result = toxicity_filter.check(text)
    status = '✅' if result.should_block == expected_toxic else '❌'
    print(f'\n{status} "{text}"')
    print(f'   Description: {description}')
    print(f'   Level: {result.level.value}')
    print(f'   Should Block: {result.should_block} (expected: {expected_toxic})')
    print(f'   Confidence: {result.confidence:.2f}')
    if result.matched_patterns:
        print(f'   Matched: {result.matched_patterns[:3]}')

🛡️ Toxicity Filter Tests:

✅ "Как получить СНИЛС?"
   Description: Обычный вопрос
   Level: safe
   Should Block: False (expected: False)
   Confidence: 1.00

✅ "Спасибо за помощь!"
   Description: Благодарность
   Level: safe
   Should Block: False (expected: False)
   Confidence: 1.00

❌ "Блин, ничего не работает"
   Description: Лёгкое недовольство
   Level: low
   Should Block: False (expected: True)
   Confidence: 0.70
   Matched: ['\\bблин\\b']

❌ "Какого хрена?!"
   Description: Грубость
   Level: safe
   Should Block: False (expected: True)
   Confidence: 1.00

❌ "Это полный бред"
   Description: Негатив
   Level: safe
   Should Block: False (expected: True)
   Confidence: 1.00


In [ ]:
# Получаем ответ для токсичного сообщения
toxic_text = 'Это полный бред'
result = toxicity_filter.check(toxic_text)
response = toxicity_filter.get_response(result)

print('📝 Toxicity Response:')
print(f'   Input: {toxic_text}')
print(f'   Response: {response}')

📝 Toxicity Response:
   Input: Это полный бред
   Response: None


---
## 🔧 Tools Testing

In [ ]:
# Загружаем доступные tools
from app.tools import ALL_TOOLS

print(f'✅ Loaded {len(ALL_TOOLS)} tools:')
for tool in ALL_TOOLS:
    print(f'   - {tool.name}: {tool.description[:60]}...')

✅ Loaded 4 tools:
   - find_nearest_mfc_tool: Найти ближайший МФЦ (Многофункциональный центр) по адресу по...
   - get_pensioner_categories_tool: Получить список категорий услуг для пенсионеров

Используй э...
   - get_pensioner_services_tool: Найти услуги для пенсионеров по району и категориям.

Исполь...
   - search_city_services: Поиск информации о государственных услугах Санкт-Петербурга....


In [ ]:
# Тестируем tool напрямую (пример с geocode)

# from app.tools.city_api import geocode_address
#
# result = geocode_address.invoke({'address': 'Невский проспект 1'})
# print('📍 Geocode Result:')
# print(result)

---
## 🔀 Agent State Testing

In [ ]:
# Импортируем состояния агентов
from langchain_core.messages import AIMessage, HumanMessage

from app.agent.state import (
    AgentState,
    create_ai_response,
    create_error_response,
    get_last_user_message,
)

print('✅ Agent states imported')

✅ Agent states imported


In [ ]:
# Тестируем извлечение сообщения
# Разные форматы content (как от LangGraph Studio)

test_states = [
    # Формат 1: простая строка
    {'messages': [HumanMessage(content='Привет')]},
    # Формат 2: list[dict] от Studio
    {'messages': [HumanMessage(content=[{'type': 'text', 'text': 'Как дела?'}])]},
    # Формат 3: пустой
    {'messages': []},
]

print('📝 get_last_user_message() Tests:')
print('=' * 50)

for i, state in enumerate(test_states, 1):
    result = get_last_user_message(state)
    print(f'{i}. Input: {state["messages"]}')
    print(f'   Output: "{result}"')
    print()

📝 get_last_user_message() Tests:
1. Input: [HumanMessage(content='Привет', additional_kwargs={}, response_metadata={})]
   Output: "Привет"

2. Input: [HumanMessage(content=[{'type': 'text', 'text': 'Как дела?'}], additional_kwargs={}, response_metadata={})]
   Output: "Как дела?"

3. Input: []
   Output: ""



In [ ]:
# Тестируем создание ответов
response = create_ai_response('Это тестовый ответ')
print('📝 create_ai_response():')
print(f'   {response}')

error_response = create_error_response(Exception('Test error'), fallback_message='Произошла ошибка')
print('\n📝 create_error_response():')
print(f'   {error_response}')

📝 create_ai_response():
   {'messages': [AIMessage(content='Это тестовый ответ', additional_kwargs={}, response_metadata={})], 'final_response': 'Это тестовый ответ'}
02:52:07 [error   ] agent_error                    app=city-assistant error='Test error'

📝 create_error_response():
   {'messages': [AIMessage(content='Произошла ошибка', additional_kwargs={}, response_metadata={})], 'final_response': 'Произошла ошибка', 'metadata': {'error': 'Test error', 'error_type': 'Exception'}}


---
## 🔄 Hybrid Agent Testing

In [ ]:
# Загружаем Hybrid Agent
from app.agent.hybrid import get_hybrid_graph

hybrid_graph = get_hybrid_graph()
print('✅ Hybrid Agent loaded')
print(f'   Nodes: {list(hybrid_graph.nodes.keys())}')

02:52:07 [info    ] hybrid_graph_build_start       app=city-assistant with_checkpointer=False
02:52:08 [info    ] hybrid_graph_build_complete    app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
✅ Hybrid Agent loaded
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response']


In [ ]:
# Sample запросы для тестирования
HYBRID_SAMPLES = [
    'Привет, как дела?',
    'Как получить СНИЛС?',
    'Какие документы нужны для загранпаспорта?',
    'Где мне получить справку о составе семьи, если я живу в Невском районе?',
    'Я живу по адресу Комендантский проспект, дом 15. Куда обращаться за оформлением социальной карты пенсионера?',
    'Привет, где ближайший МФЦ по адресу Большевиков дом 68 корпус 1'
    'Где ближайшая поликлиника, если я живу по адресу: проспект Славы, дом 12?',
    'Какой ближайший МФЦ к Большевиков дом 68 корпус 1',
]

print('📝 Hybrid Agent Sample Queries:')
for i, q in enumerate(HYBRID_SAMPLES, 1):
    print(f'   {i}. {q}')

📝 Hybrid Agent Sample Queries:
   1. Привет, как дела?
   2. Как получить СНИЛС?
   3. Какие документы нужны для загранпаспорта?
   4. Где мне получить справку о составе семьи, если я живу в Невском районе?
   5. Я живу по адресу Комендантский проспект, дом 15. Куда обращаться за оформлением социальной карты пенсионера?
   6. Привет, где ближайший МФЦ по адресу Большевиков дом 68 корпус 1Где ближайшая поликлиника, если я живу по адресу: проспект Славы, дом 12?
   7. Какой ближайший МФЦ к Большевиков дом 68 корпус 1


In [ ]:
# Тестируем Hybrid Agent
from langchain_core.messages import HumanMessage

query = HYBRID_SAMPLES[2]
print(f'🤖 Hybrid Agent Test: "{query}"')
print('=' * 60)

initial_state = {
    'messages': [HumanMessage(content=query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

# Запускаем агент
result = hybrid_graph.invoke(initial_state)

print('\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   Messages count: {len(result.get("messages", []))}')

# Последнее сообщение (ответ агента)
if result.get('messages'):
    last_msg = result['messages'][-1]
    print('\n💬 Agent Response:')
    print(f'   Type: {type(last_msg).__name__}')
    content = last_msg.content if hasattr(last_msg, 'content') else str(last_msg)
    print(
        f'   Content: {content[:500]}...' if len(str(content)) > 500 else f'   Content: {content}'
    )

🤖 Hybrid Agent Test: "Какие документы нужны для загранпаспорта?"
02:52:42 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=41
02:52:42 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='какие документы нужны для загранпаспорта?'
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=ecc64211-337d-417b-922a-58bdafc419e0,id=ecc64211-337d-417b-922a-58bdafc419e0; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=34e43bf9-1396-4014-b5ed-c50fc7f16986; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=d004ecc8-69cd-4d06-b8cc-60dbffd0e758; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=d004ecc8-69cd-4d06-b8cc-60dbffd0e758; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=34e43bf9-1396-4014-b5ed-c50fc7f16986; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=b4941ac9-ce23-4ae7-93d8-3b5e0315a257; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=46c18826-9b2f-4938-a610-fc00f35ccd68; trace=ecc64211-337d-417b-922a-

Sending compressed multipart request with context: trace=ecc64211-337d-417b-922a-58bdafc419e0,id=b6977172-0869-400f-830f-ed9096cf2e9d; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=6ebbfe6d-c6cf-42d5-86e5-b7f21ddbe6c6; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=6ebbfe6d-c6cf-42d5-86e5-b7f21ddbe6c6; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=9e7eae8f-0e5c-468b-ad34-d44545309878; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=cdb6d1ec-2f8f-4e5d-a14c-c082b48182dd; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=10386242-a3ad-4f9a-b7cc-265f055bdfbe; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=10386242-a3ad-4f9a-b7cc-265f055bdfbe; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=9e4ba6da-dac6-4af1-b675-d8e058d66edb; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=d3b5ca91-bea9-4666-b511-33f7bfde536a; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=e63583ba-82bc-4677-b471-8d736ce45cc2; trace=ecc64211-337d-417b-922a-58bdafc419e0,id=e63583ba-82bc-4677-b471-8d736ce45cc2; trace=ecc64211-337d-417b-

In [ ]:
# Тестируем с токсичным запросом
toxic_query = 'Да что за ерунда такая?!'
print(f'🤖 Hybrid Agent Toxicity Test: "{toxic_query}"')
print('=' * 60)

toxic_state = {
    'messages': [HumanMessage(content=toxic_query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

result = hybrid_graph.invoke(toxic_state)

print('\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   toxicity_response: {result.get("toxicity_response")}')

🤖 Hybrid Agent Toxicity Test: "Да что за ерунда такая?!"
02:52:15 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=24
02:52:15 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='да что за ерунда такая?!'
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=37807daf-9a26-4964-b91a-7a599aa58330; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=81e7270e-cde7-4ec8-b5b6-2944f4cecf08; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=ec4185ba-9936-4565-a65f-650d4e89ad05; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=70a9e65b-cfff-48e8-b138-7aa7bd83d7cd; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=70a9e65b-cfff-48e8-b138-7aa7bd83d7cd; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=a25713e3-29e5-4b25-b70c-51ecd78bceec; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=cf57a050-de4d-4145-9dad-96c9c3d537a2; trace=0b752e88-8974-4e1f-86b8-00da80a7a0b2,id=9f587ceb-

---
## 🎯 Supervisor Agent Testing

In [ ]:
# Загружаем Supervisor Agent
from app.agent.supervisor import get_supervisor_graph

supervisor_graph = get_supervisor_graph()
print('✅ Supervisor Agent loaded')
print(f'   Nodes: {list(supervisor_graph.nodes.keys())}')

02:52:17 [info    ] supervisor_graph_build_start   app=city-assistant with_checkpointer=False
02:52:17 [info    ] supervisor_graph_build_complete app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
✅ Supervisor Agent loaded
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response']


In [ ]:
# Sample запросы для разных интентов
SUPERVISOR_SAMPLES = [
    ('Привет!', 'greeting'),
    ('Как оформить паспорт?', 'rag_query'),
    ('Найди адрес Невский 1', 'api_call'),
    ('Спасибо, до свидания', 'farewell'),
]

print('📝 Supervisor Agent Sample Queries:')
for query, expected_intent in SUPERVISOR_SAMPLES:
    print(f'   "{query}" → {expected_intent}')

📝 Supervisor Agent Sample Queries:
   "Привет!" → greeting
   "Как оформить паспорт?" → rag_query
   "Найди адрес Невский 1" → api_call
   "Спасибо, до свидания" → farewell


In [ ]:
# Тестируем Supervisor Agent
query, expected_intent = SUPERVISOR_SAMPLES[1]  # rag_query

print(f'🎯 Supervisor Test: "{query}"')
print(f'   Expected intent: {expected_intent}')
print('=' * 60)

initial_state = {
    'messages': [HumanMessage(content=query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
    # Supervisor-specific
    'next_agent': None,
    'rag_context': None,
}

result = supervisor_graph.invoke(initial_state)

print('\n📊 Result:')
print(f'   intent: {result.get("intent")}')
print(f'   intent_confidence: {result.get("intent_confidence")}')
print(f'   next_agent: {result.get("next_agent")}')

# Ответ
if result.get('messages'):
    last_msg = result['messages'][-1]
    content = last_msg.content if hasattr(last_msg, 'content') else str(last_msg)
    print('\n💬 Response:')
    print(f'   {content[:300]}...' if len(str(content)) > 300 else f'   {content}')

🎯 Supervisor Test: "Как оформить паспорт?"
   Expected intent: rag_query
02:52:17 [info    ] supervisor_node                app=city-assistant node=check_toxicity query_length=21
02:52:17 [debug   ] toxicity_passed                app=city-assistant
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=4a1c506b-158e-4c30-b27b-360c4162378b,id=0be76fdb-9a93-451a-b39e-7a2ebe86b505; trace=4a1c506b-158e-4c30-b27b-360c4162378b,id=d73a65d2-356e-4f65-8213-f48980d5eceb; trace=4a1c506b-158e-4c30-b27b-360c4162378b,id=5761fbc5-1867-48c5-8783-c4afec2d3ef5; trace=4a1c506b-158e-4c30-b27b-360c4162378b,id=5761fbc5-1867-48c5-8783-c4afec2d3ef5; trace=4a1c506b-158e-4c30-b27b-360c4162378b,id=4a1c506b-158e-4c30-b27b-360c4162378b; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=045cf60f-4541-4ee7-8874-9b53113b4e5b; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=39c6ba57-1b3e-4cd9-859b-0346b896c0be; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=b5d47075-62da-4461-823a-009aa0ec235c; trace=045

---
## 📜 Prompts Inspection

In [ ]:
# Смотрим системные промпты агентов

print('📜 Agent Prompts:')
print('=' * 60)

# Hybrid Agent prompt
try:
    from app.agent.hybrid import SYSTEM_PROMPT as HYBRID_PROMPT

    print('\n🔄 Hybrid Agent System Prompt:')
    print('-' * 40)
    print(HYBRID_PROMPT[:500] + '...' if len(HYBRID_PROMPT) > 500 else HYBRID_PROMPT)
except ImportError:
    print('   Hybrid prompt not exported')

# RAG prompts
try:
    from app.rag.enhancers import QueryRewriter

    rewriter = QueryRewriter()
    print('\n📝 Query Rewriter Prompt:')
    print('-' * 40)
    # Если есть доступ к промпту
except Exception as e:
    print(f'   Could not load: {e}')

📜 Agent Prompts:
   Hybrid prompt not exported

📝 Query Rewriter Prompt:
----------------------------------------


---
## 🧪 LangGraph Studio Format Testing

Тестирование форматов сообщений от LangGraph Studio

In [ ]:
# Симулируем формат от LangGraph Studio
from langchain_core.messages import HumanMessage

# Studio отправляет content как list[dict]
studio_message = HumanMessage(content=[{'type': 'text', 'text': 'Как получить паспорт?'}])

studio_state = {
    'messages': [studio_message],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

# Извлекаем текст
extracted = get_last_user_message(studio_state)
print('🖥️ LangGraph Studio Format Test:')
print(f'   Raw content: {studio_message.content}')
print(f'   Extracted text: "{extracted}"')

🖥️ LangGraph Studio Format Test:
   Raw content: [{'type': 'text', 'text': 'Как получить паспорт?'}]
   Extracted text: "Как получить паспорт?"


In [ ]:
# Тестируем агент с Studio-форматом
print('🖥️ Hybrid Agent with Studio Format:')
print('=' * 60)

result = hybrid_graph.invoke(studio_state)

print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   Messages: {len(result.get("messages", []))}')

if result.get('messages'):
    last = result['messages'][-1]
    print(
        f'\n💬 Response: {last.content[:200]}...' if hasattr(last, 'content') else str(last)[:200]
    )

🖥️ Hybrid Agent with Studio Format:
02:52:23 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=21
02:52:23 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='как получить паспорт?'
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=6c67a1d3-9ae3-4fe4-8f59-166b50959566; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=b337c0e5-8fc8-4118-b9e4-56e1572096d7; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=b337c0e5-8fc8-4118-b9e4-56e1572096d7; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=4998094b-2dee-4f6f-9a44-912f33ba95c0; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=deb836a2-1d1e-4736-84e3-86f19443937b; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=68c2eb0f-302a-4e71-9806-3428a292d340; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=68c2eb0f-302a-4e71-9806-3428a292d340; trace=045cf60f-4541-4ee7-8874-9b53113b4e5b,id=04ee92d8-934f-4112-aed6-167e83064

---
## 📊 Batch Agent Testing

In [ ]:
# Batch тестирование Hybrid Agent
all_samples = [
    'Привет!',
    'Как получить СНИЛС?',
    'Какие льготы есть для пенсионеров?',
    'Спасибо за помощь!',
]

print('📊 Batch Hybrid Agent Tests')
print('=' * 60)

for query in all_samples:
    state = {
        'messages': [HumanMessage(content=query)],
        'is_toxic': False,
        'toxicity_response': None,
        'intent': '',
        'intent_confidence': 0.0,
        'tool_result': None,
        'final_response': None,
        'metadata': {},
    }

    try:
        result = hybrid_graph.invoke(state)
        last_msg = result['messages'][-1] if result.get('messages') else None
        response = last_msg.content[:100] if last_msg and hasattr(last_msg, 'content') else 'N/A'
        print(f'\n🔍 "{query}"')
        print(f'   Response: {response}...')
    except Exception as e:
        print(f'\n❌ "{query}"')
        print(f'   Error: {e}')

📊 Batch Hybrid Agent Tests
02:52:26 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=7
02:52:26 [info    ] hybrid_node                    app=city-assistant node=classify_intent query=привет!
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=757b1bbf-5f5e-41a2-b443-270d4e40eaf0; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=58e79db8-4c6c-4ef5-880e-0af428f8c288; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=58e79db8-4c6c-4ef5-880e-0af428f8c288; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=67b447cb-e1d3-4ceb-bd22-4b35f45a7acb; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=b3eb9383-0c1b-42e2-ad3a-445577b7dd73; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=17dbd29c-d852-4753-be2d-19b5e7864d37; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=17dbd29c-d852-4753-be2d-19b5e7864d37; trace=de149b6a-ac2b-4ec8-a8fd-608a3433c7e2,id=e4e76109-3bef-4a51-a137-c4ad5eaecf29; trace=de149b6a-ac2b-4

---
## 🔗 Full Pipeline Test (graphs.py exports)

In [ ]:
# Проверяем экспорты из graphs.py (для langgraph.json)
from app.graphs import hybrid, rag, supervisor

# Это функции-фабрики, вызываем их для получения графов
hybrid_g = hybrid()
supervisor_g = supervisor()
rag_g = rag()

print('✅ All graphs exported from app.graphs:')
print(f'   hybrid: {type(hybrid_g).__name__}')
print(f'   supervisor: {type(supervisor_g).__name__}')
print(f'   rag: {type(rag_g).__name__}')

02:52:37 [info    ] graph_factory_hybrid           app=city-assistant config_present=False
02:52:37 [info    ] hybrid_graph_build_start       app=city-assistant with_checkpointer=False
02:52:37 [info    ] hybrid_graph_build_complete    app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
02:52:37 [info    ] graph_factory_supervisor       app=city-assistant config_present=False
02:52:37 [info    ] supervisor_graph_build_start   app=city-assistant with_checkpointer=False
02:52:37 [info    ] supervisor_graph_build_complete app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
02:52:37 [info    ] graph_factory_rag              app=city-assistant config_present=False
02:52:37 [info    ] graph_build_start              app=city-assistant

In [ ]:
# Финальный интеграционный тест
print('🎯 Final Integration Test')
print('=' * 60)

test_query = 'Какие документы нужны для регистрации ребёнка?'

state = {
    'messages': [HumanMessage(content=test_query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

print(f'Query: "{test_query}"')
print()

result = hybrid_graph.invoke(state)

print('Result:')
print(f'  is_toxic: {result.get("is_toxic")}')
print(f'  messages: {len(result.get("messages", []))}')

if result.get('messages'):
    last = result['messages'][-1]
    if hasattr(last, 'content'):
        print('\n💬 Final Response:')
        print(last.content)

🎯 Final Integration Test
Query: "Какие документы нужны для регистрации ребёнка?"

02:52:37 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=46
02:52:37 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='какие документы нужны для регистрации ребёнка?'
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=8c12975a-3c74-4d8b-8bf3-4c4d69467705,id=405199ca-aa6d-4e42-91d6-71dec8fde44c; trace=8c12975a-3c74-4d8b-8bf3-4c4d69467705,id=06b5d69a-e636-47e8-8c2e-55e435442b92; trace=8c12975a-3c74-4d8b-8bf3-4c4d69467705,id=eb50e81c-895a-43ff-ae33-a636edee3d83; trace=8c12975a-3c74-4d8b-8bf3-4c4d69467705,id=eb50e81c-895a-43ff-ae33-a636edee3d83; trace=8c12975a-3c74-4d8b-8bf3-4c4d69467705,id=8c12975a-3c74-4d8b-8bf3-4c4d69467705; trace=23baa65e-3643-45da-8f40-04238c00938b,id=23baa65e-3643-45da-8f40-04238c00938b; trace=23baa65e-3643-45da-8f40-04238c00938b,id=73ba4a79-50ef-4333-bff9-a9fa35a1a3c1; trace=23

Sending compressed multipart request with context: trace=23baa65e-3643-45da-8f40-04238c00938b,id=42070a02-8c5e-4f64-b40d-cb3dc0a602a7; trace=23baa65e-3643-45da-8f40-04238c00938b,id=dbefc6f0-1aea-4d78-88a3-147164fb3171; trace=23baa65e-3643-45da-8f40-04238c00938b,id=dbefc6f0-1aea-4d78-88a3-147164fb3171; trace=23baa65e-3643-45da-8f40-04238c00938b,id=c65b335e-bf7f-4c88-aca7-9f2a6eb1bca3; trace=23baa65e-3643-45da-8f40-04238c00938b,id=2786a2cb-5dd6-4b36-8160-6cf91b56176e; trace=23baa65e-3643-45da-8f40-04238c00938b,id=f3079ad9-506f-452c-89c2-f154cbdb7172; trace=23baa65e-3643-45da-8f40-04238c00938b,id=f3079ad9-506f-452c-89c2-f154cbdb7172; trace=23baa65e-3643-45da-8f40-04238c00938b,id=a2078bfd-8e0b-4db8-bed1-55cc16e89de2; trace=23baa65e-3643-45da-8f40-04238c00938b,id=6c3846df-9ec3-4dd7-85ee-5f554b968125; trace=23baa65e-3643-45da-8f40-04238c00938b,id=4883cc41-8518-4b67-a976-cc21aae8dbbf; trace=23baa65e-3643-45da-8f40-04238c00938b,id=4883cc41-8518-4b67-a976-cc21aae8dbbf; trace=23baa65e-3643-45da-